In [1]:
 #Import All Libraries Here
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split , RandomizedSearchCV


from imblearn.over_sampling import SMOTE 
from imblearn.over_sampling import RandomOverSampler


from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import optuna

import gensim

# PyTorch Import

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# Record Constants
INPUT_TRAIN_IN = "../../../data/train_in.csv"
INPUT_TRAIN_OUT = "../../../data/train_out.csv"  
INPUT_TEST_IN = "../../../data/test_in.csv"
INPUT_TEST_OUT = "../../../data/test_out.csv"
INPUT_VALIDATION_IN = "../../../data/valid_in_nucleo.csv"
INPUT_VALIDATION_OUT  = "../../../data/valid_out.csv"

TARGET_MODEL_PATH = '../../webapp/model_files'


# 1 - One Hot Encoding with Pytorch in build Emnedding 
# 2 - K-mers with Word2Vec
ENCODING_METHOD = 1


# 1 - LSTM with Cross Entropy 
MODEL = 1


FRAMEWORK = "PYTORCH"

# Startegy to Crop Sequene
# MID - Modification is present at Mid of cropped Sequence 
# END - Modification is present at End of cropepd Sequence 
CROP_STRATEGY = 'END'

# Y Category Encoding Method
# LABEL or ONE_HOT 
TARGET_ENCODING = 'LABEL'



c:\Users\shashi.vish\Python Environment\RNA_ModX\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv(INPUT_TRAIN_IN, header=None , skiprows=1 )
y_train_raw =  pd.read_csv(INPUT_TRAIN_OUT, header=None , skiprows=1)

x_test_raw =  pd.read_csv(INPUT_TEST_IN, header=None , skiprows=1 )
y_test_raw =  pd.read_csv(INPUT_TEST_OUT, header=None , skiprows=1)

x_valid_raw =  pd.read_csv(INPUT_VALIDATION_IN, header=None , skiprows=1 )
y_valid_raw =  pd.read_csv(INPUT_VALIDATION_OUT, header=None , skiprows=1 )



In [3]:
print(f"Train Shape of X : {x_train_raw.shape} and Tranin Shape of Y : {x_train_raw.shape}")
print(f"Test Shape of X : {x_test_raw.shape} and Test Shape of Y : {y_test_raw.shape}")
print(f"Validation Shape of X : {x_valid_raw.shape} and Validation Shape of Y : {y_valid_raw.shape}")

Train Shape of X : (304661, 1001) and Tranin Shape of Y : (304661, 1001)
Test Shape of X : (1200, 1001) and Test Shape of Y : (1200, 12)
Validation Shape of X : (3599, 1001) and Validation Shape of Y : (3599, 12)


### Encode with One Hot Encoding

In [4]:
x_train_split_51 = []
y_train_split_3 = []
x_test_split_51 = []
y_test_split_3 = []
x_valid_split_51 = []
y_valid_split_3 = []

window_size = 1000
target_size = 1

def splitSequenceIntoXAndY(sequence , TYPE):


    for i in range(len(sequence) - window_size - target_size + 1):
        subsequence = sequence[i:i+window_size]
        target = sequence[i+window_size:i+window_size+target_size]

        if TYPE == 'TRAIN':
            x_train_split_51.append(subsequence)
            y_train_split_3.append(target)
        if TYPE == 'TRAIN':
            x_test_split_51.append(subsequence)
            y_test_split_3.append(target)
        if TYPE == 'TRAIN':
            x_valid_split_51.append(subsequence)
            y_valid_split_3.append(target)            

def encode_with_one_hot_encoding(x_train_raw , TYPE):
    concatenated_column= x_train_raw.apply(lambda row: ''.join(map(str, row)), axis=1)
    df_result = x_train_raw.assign(Sequence=concatenated_column)
    df_result['Sequence'].apply(splitSequenceIntoXAndY , args =(TYPE,))


In [5]:
encode_with_one_hot_encoding(x_train_raw ,"TRAIN")
encode_with_one_hot_encoding(x_test_raw ,"TEST")
encode_with_one_hot_encoding(x_valid_raw ,"VALID")

In [10]:
y_train_split_3

['T',
 'A',
 'T',
 'C',
 'G',
 'T',
 'G',
 'C',
 'A',
 'T',
 'T',
 'C',
 'A',
 'A',
 'T',
 'C',
 'T',
 'A',
 'G',
 'G',
 'C',
 'G',
 'G',
 'T',
 'A',
 'A',
 'G',
 'A',
 'T',
 'A',
 'T',
 'C',
 'G',
 'T',
 'G',
 'C',
 'A',
 'G',
 'T',
 'A',
 'A',
 'A',
 'G',
 'T',
 'G',
 'T',
 'G',
 'T',
 'C',
 'G',
 'G',
 'A',
 'C',
 'A',
 'C',
 'G',
 'T',
 'C',
 'C',
 'G',
 'A',
 'A',
 'T',
 'C',
 'G',
 'A',
 'A',
 'A',
 'A',
 'G',
 'C',
 'G',
 'A',
 'G',
 'T',
 'G',
 'A',
 'G',
 'T',
 'C',
 'C',
 'A',
 'T',
 'G',
 'T',
 'A',
 'G',
 'G',
 'G',
 'T',
 'G',
 'A',
 'G',
 'A',
 'A',
 'C',
 'A',
 'C',
 'T',
 'T',
 'C',
 'C',
 'T',
 'T',
 'G',
 'G',
 'A',
 'C',
 'C',
 'C',
 'G',
 'T',
 'T',
 'A',
 'G',
 'T',
 'A',
 'T',
 'A',
 'G',
 'C',
 'A',
 'C',
 'A',
 'T',
 'G',
 'C',
 'A',
 'A',
 'A',
 'A',
 'T',
 'T',
 'A',
 'T',
 'G',
 'A',
 'T',
 'A',
 'G',
 'C',
 'T',
 'C',
 'A',
 'G',
 'G',
 'C',
 'G',
 'A',
 'C',
 'T',
 'T',
 'C',
 'G',
 'A',
 'G',
 'C',
 'G',
 'T',
 'T',
 'T',
 'T',
 'C',
 'G',
 'G',
 'C',
 'C'

In [6]:
def encode_seq(sequence):

    # A 1 0 0 0
    # C 0 1 0 0
    # T/U 0 0 0 1
    # G 0 0 1 0
    # N 0 0 0 0

    encoding_dict = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'U': [0, 0, 0, 1],
        'N': [0, 0, 0, 0],
    }

    encoded_sequence = []
    for  base in sequence:
        encoded_sequence.append(encoding_dict[base])
    return np.array(encoded_sequence).flatten()

In [7]:
print(f"Total Training Length for Encoding : {len(x_train_split_51)}")
X_train = []
X_test = []
X_valid = []
for sequence in x_train_split_51 :
    X_train.append(encode_seq(sequence))
for sequence in x_test_split_51 :
    X_test.append(encode_seq(sequence))
for sequence in x_valid_split_51 :
    X_valid.append(encode_seq(sequence))
X_train = torch.tensor(X_train , dtype=torch.long)
X_test = torch.tensor(X_test , dtype=torch.long)
X_valid = torch.tensor(X_valid , dtype=torch.long)
X_test

Total Training Length for Encoding : 243119478


In [13]:
le = LabelEncoder()
y_train = le.fit_transform(y_train_split_3)
y_test = le.fit_transform(y_test_split_3)
y_valid = le.fit_transform(y_valid_split_3)

# y_train = []
# y_test =[]
# y_valid = []
# for sequence in y_train_split_3 :
#     y_train.append(encode_seq(sequence))
# for sequence in y_test_split_3 :
#     y_test.append(encode_seq(sequence))
# for sequence in y_valid_split_3 :
#     y_valid.append(encode_seq(sequence))
y_train = torch.tensor(y_train , dtype=torch.long)
y_test = torch.tensor(y_test , dtype=torch.long)
y_valid = torch.tensor(y_valid , dtype=torch.long)
y_train

tensor([4, 0, 4,  ..., 3, 3, 3])

In [ ]:
hyperparameter = {}

hyperparameter['INPUT_DIMENSION'] = 4 # For One Hot Encoding Input Dimension would be 4 as there only 4 unique nucleocide 

hyperparameter['HIDDEN_DIMENSION'] = 32
hyperparameter['NO_OF_LAYERS'] = 3
hyperparameter['BATCH_SIZE'] = 32
hyperparameter['OUTPUT_DIMENSION'] = len(y_train[0])
hyperparameter['EMBEDDING_DIMENSION'] = 32 # if you are using Word2Vec Encoding then this should be same as Word2Vec Embedding Dim 
hyperparameter['DROP_OUT'] = 0.5

In [ ]:
class RNADataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

train_dataset = RNADataset(X_train, y_train)
test_dataset = RNADataset(X_test, y_test)
valid_dataset = RNADataset(X_valid, y_valid)


train_dataloader = DataLoader(train_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=hyperparameter['BATCH_SIZE'], shuffle=False)

In [ ]:

class RNATransferLearning(nn.Module):
    def __init__(self, input_dim,embedding_dim, hidden_dim,num_layers, output_dim , dropout=0.5):
        super(RNATransferLearning, self).__init__()

        #Pytroch Embedding 
        self.embedding = nn.Embedding(input_dim,embedding_dim)

        #Bi-Directional LSTM Model
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers, dropout= dropout, batch_first=True )

        #Fully Connected Layer   
        self.fc = nn.Linear(hidden_dim, output_dim) 

        # Drop out layer for Overfitting 
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):

        x  = x.long()

        x_embeded =  self.embedding(x)
        
        lstm_out, (h,c) = self.lstm(x_embeded)

        # Pass it to drop out layer 
        context_vector = self.dropout(h[-1])
        
        # Finally pass it to fully connected layer.
        out = self.fc(context_vector)
        
        return out # Sequeez to Single dimension for loss calculation
    


In [ ]:
def validate_model(model, test_dataloader , device ,loss_function):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.float().to(device)

            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            running_loss += loss.item()

    validation_loss = running_loss / len(test_dataloader)

    return validation_loss


def train_model(model , train_dataloader , test_dataloader ,device ,epochs, optimizer ,loss_function):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_dataloader):

            inputs  = inputs.to(device)
            labels = labels.float().to(device)

            optimizer.zero_grad()
            outputs = model(inputs) 
            
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        epoch_loss = running_loss / len(train_dataloader)
        val_loss = validate_model(model, test_dataloader ,device, loss_function)
        print(f"Epoch {epoch + 1}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")



In [ ]:
model = RNATransferLearning(input_dim=hyperparameter['INPUT_DIMENSION'],
                            embedding_dim=hyperparameter['EMBEDDING_DIMENSION'], 
                            hidden_dim=hyperparameter['HIDDEN_DIMENSION'] , 
                            num_layers = hyperparameter['NO_OF_LAYERS'],
                            output_dim=hyperparameter['OUTPUT_DIMENSION'],
                            dropout=hyperparameter['DROP_OUT'] )


loss_function = nn.CrossEntropyLoss()  ## MSELoss of Regression problem  # BCELoss for binary classification
optimizer = optim.Adam(model.parameters() ,  lr=0.01)

# Number of Parameters for Model
total_parameters = []
for p in model.parameters():
    total_parameters.append(p.numel())

print(f"Total Number of Parameters for Model Training : { sum(total_parameters)} " )

# Train the model
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Model Parameters  : " , hyperparameter)

# Train Model with configured Parameter
train_model(model, train_dataloader ,test_dataloader, device ,num_epochs,optimizer,loss_function)


Total Number of Parameters for Model Training : 345 
Model Parameters  :  {'INPUT_DIMENSION': 3, 'HIDDEN_DIMENSION': 3, 'NO_OF_LAYERS': 3, 'BATCH_SIZE': 32, 'OUTPUT_DIMENSION': 12, 'EMBEDDING_DIMENSION': 3, 'DROP_OUT': 0.5}
Epoch 1, Train Loss: 7.4389, Val Loss: 7.4289
Epoch 2, Train Loss: 7.4303, Val Loss: 7.4302
Epoch 3, Train Loss: 7.4306, Val Loss: 7.4280
Epoch 4, Train Loss: 7.4290, Val Loss: 7.4275
Epoch 5, Train Loss: 7.4293, Val Loss: 7.4238
Epoch 6, Train Loss: 7.4264, Val Loss: 7.4206
Epoch 7, Train Loss: 7.4247, Val Loss: 7.4138
Epoch 8, Train Loss: 7.4209, Val Loss: 7.4076
Epoch 9, Train Loss: 7.4183, Val Loss: 7.4088
Epoch 10, Train Loss: 7.4182, Val Loss: 7.4116
Epoch 11, Train Loss: 7.4167, Val Loss: 7.4044
Epoch 12, Train Loss: 7.4177, Val Loss: 7.4045
Epoch 13, Train Loss: 7.4185, Val Loss: 7.4051
Epoch 14, Train Loss: 7.4183, Val Loss: 7.4063


KeyboardInterrupt: 

In [ ]:
window_size = 51
target_size = 3
model_name = "model_feature_windoe_"+str(window_size)+"_target_"+str(target_size)
torch.save(model.state_dict, model_name)